In [2]:
import numpy as np
import pandas as pd 
import os

In [3]:
import zipfile
zip_file = "/kaggle/input/platesv2/plates.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
  zip_ref.extractall()  # extract current working directory

In [4]:
import torch
from torchvision import transforms

data_dir = "/kaggle/working/plates"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"

labels = os.listdir(train_dir)[:2]
labels

['dirty', 'cleaned']

In [5]:
image = []
image_label = []

In [9]:
import cv2
from PIL import Image
for label in labels:
    path = os.path.join(train_dir, label)
    for image_path in os.listdir(path):
        if image_path == ".DS_Store":
            continue
#         print(image_path)
        curr_image = Image.open(os.path.join(path, image_path))
#         curr_image = cv2.imread(os.path.join(path, image_path))
        image.append(curr_image)
        image_label.append(label)

In [10]:
len(image), len(image_label)

(76, 76)

In [12]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(), 
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # normalize pixel values
])

In [21]:
from torch.utils.data import Dataset
class my_dataset(Dataset):
    def __init__(self, images, labels, transform):
        self.images = images # these are features
        self.labels = labels
        self.transform = transform
    def __getitem__(self, item):
        image = self.images[item]
        image_tranformed = self.transform(image)
        label = self.labels[item]
        return image_tranformed, label
    def __len__(self):
        return len(self.labels)
        

In [22]:
data = my_dataset(image, image_label, transform)
data

In [23]:
from torch.utils.data import DataLoader
data_loader = DataLoader(data, batch_size = 20, shuffle=True)

In [29]:
from torch import nn
import torch.nn.functional as F
class kaggle_plate_model(nn.Module):
    def __init__(self):
        super(kaggle_plate_model, self).__init__()
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x) 

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [30]:
model = kaggle_plate_model()
model

kaggle_plate_model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [31]:
from torch import optim
lr = 0.001
epoch = 100
optimizer = optim.Adam(model.parameters(), lr)
criterion = nn.CrossEntropyLoss()

In [32]:
from tqdm.auto import tqdm

In [33]:
for i in tqdm(range(epoch)):
    for image, label in data_loader:
        
        optimizer.zero_grad()
        
        # forward pass
        pred = model(image)
        loss = criterion(pred, label)
        
        # backward pass
        loss.backward()
        optimizer.step()
    print(f"Epoch : {i+1}, Loss : {loss}")

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: shape '[-1, 1568]' is invalid for input of size 2621440